# Batches v2

In [50]:
import tensorflow as tf

def process_path(paths, labels):
    features = []
    # load waveforms in memory
    for path in paths:
        raw_audio = tf.io.read_file(path)
        waveform, sr = tf.audio.decode_wav(raw_audio)
        features.append(waveform)

#     tf.print(features[0], len(features[0]), len(features[1]))
    features = tf.keras.preprocessing.sequence.pad_sequences(features, dtype='float', padding='post', truncating='post')
#     tf.print(features[0], len(features[0]), len(features[1]))
#     tf.print(features)
    return features, labels

def tf_function_wrapper(features, labels):
    return tf.py_function(func=process_path, inp=[features['filename'], labels], Tout=tf.float32), labels

librivox_paths_ds = tf.data.experimental.make_csv_dataset(
    "librivox-test-clean-wav.csv", batch_size=2,
    shuffle=False, label_name="transcript")

librivox_ds = librivox_paths_ds.map(tf_function_wrapper)

for feature_batch, labels in librivox_ds.take(1):
    tf.print(feature_batch)

    

[[[0.000732421875]
  [0.000823974609]
  [0.00173950195]
  ...
  [0]
  [0]
  [0]]

 [[-0.00778198242]
  [-0.00921630859]
  [-0.00909423828]
  ...
  [-0.00262451172]
  [-0.00671386719]
  [-0.0113830566]]]


In [51]:
for feature_batch in librivox_ds.take(1):
    tf.print(feature_batch[0][1])
    for key, valeu in feature_batch.numpy():
        print(key)
#     for key, valeu in feature_batch.numpy():
#         print(value)

[[-0.00778198242]
 [-0.00921630859]
 [-0.00909423828]
 ...
 [-0.00262451172]
 [-0.00671386719]
 [-0.0113830566]]


AttributeError: 'tuple' object has no attribute 'numpy'

In [91]:
import tensorflow as tf
t = tf.constant([[1, 2, 3], [4, 5, 6]])
paddings = tf.constant([[0, 0], [0, 2]])
# 'constant_values' is 0.
# rank of 't' is 2.
a = tf.pad(t, paddings, "CONSTANT")  # [[0, 0, 0, 0, 0, 0, 0],
                                 #  [0, 0, 1, 2, 3, 0, 0],
                                 #  [0, 0, 4, 5, 6, 0, 0],
                                 #  [0, 0, 0, 0, 0, 0, 0]]
a

<tf.Tensor: shape=(2, 5), dtype=int32, numpy=
array([[1, 2, 3, 0, 0],
       [4, 5, 6, 0, 0]], dtype=int32)>

In [54]:
from soundfile import read
path = '/Users/gabriel.t.nishimura/projects/masters/ctc/data_dir/LibriSpeech/test-clean-new/61-70970-0000.wav'
read(path)

(array([ 0.00073242,  0.00082397,  0.0017395 , ..., -0.00631714,
        -0.00570679, -0.00668335]),
 16000)

In [70]:
import tensorflow as tf
raw_audio = tf.io.read_file(path)
waveform = tf.audio.decode_wav(raw_audio)
transwav = tf.transpose(waveform[0])
stfts = tf.signal.stft(transwav, frame_length=1024, frame_step=256, fft_length=1024)
spectrograms = tf.abs(stfts)

num_spectrogram_bins = stfts.shape[-1]
lower_edge_hertz, upper_edge_hertz, num_mel_bins = 80.0, 7600.0, 80

linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
  num_mel_bins, num_spectrogram_bins, sample_rate, lower_edge_hertz,
  upper_edge_hertz)

mel_spectrograms = tf.tensordot(
  spectrograms, linear_to_mel_weight_matrix, 1)

mel_spectrograms.set_shape(spectrograms.shape[:-1].concatenate(
  linear_to_mel_weight_matrix.shape[-1:]))

# Compute a stabilized log to get log-magnitude mel-scale spectrograms.
log_mel_spectrograms = tf.math.log(mel_spectrograms + 1e-6)

# Compute MFCCs from log_mel_spectrograms and take the first 13.
mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrograms)
mfccs

<tf.Tensor: shape=(1, 376, 80), dtype=float32, numpy=
array([[[-3.6905033e+01,  2.3514066e+00,  3.6240559e+00, ...,
         -1.0703679e-01,  1.9348332e-01,  4.8363358e-01],
        [-3.6752304e+01,  2.2840919e+00,  3.9529059e+00, ...,
         -1.2098748e-01,  3.1118006e-01,  5.0013596e-01],
        [-3.7614742e+01,  2.1445134e+00,  3.5887289e+00, ...,
         -1.1431646e-01,  4.0307066e-01,  2.9037148e-01],
        ...,
        [-1.7701191e+01, -1.3168314e+00,  2.0635312e+00, ...,
         -2.7517197e-01, -2.6096088e-01, -2.0862819e-01],
        [-1.9440060e+01, -6.5816146e-01,  1.6152061e+00, ...,
          2.1513538e-02, -2.3255355e-03, -2.9484089e-02],
        [-2.0488327e+01, -3.4209549e-01,  5.2042776e-01, ...,
          9.0547338e-02,  9.2801861e-02,  3.1357277e-02]]], dtype=float32)>

In [78]:
import tensorflow as tf

def process_path(paths, labels):
    features = []
    # load waveforms in memory
    for path in paths:
        raw_audio = tf.io.read_file(path)
        waveform, sr = tf.audio.decode_wav(raw_audio)
        transwav = tf.transpose(waveform)
        stfts = tf.signal.stft(transwav, frame_length=1024, frame_step=256, fft_length=1024)
        spectrograms = tf.abs(stfts)

        num_spectrogram_bins = stfts.shape[-1]
        lower_edge_hertz, upper_edge_hertz, num_mel_bins = 80.0, 7600.0, 80

        linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
          num_mel_bins, num_spectrogram_bins, sample_rate, lower_edge_hertz,
          upper_edge_hertz)

        mel_spectrograms = tf.tensordot(
          spectrograms, linear_to_mel_weight_matrix, 1)

        mel_spectrograms.set_shape(spectrograms.shape[:-1].concatenate(
          linear_to_mel_weight_matrix.shape[-1:]))

        # Compute a stabilized log to get log-magnitude mel-scale spectrograms.
        log_mel_spectrograms = tf.math.log(mel_spectrograms + 1e-6)

        # Compute MFCCs from log_mel_spectrograms and take the first 13.
        mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrograms)
        mfccs
        features.append(waveform)

    features = tf.keras.preprocessing.sequence.pad_sequences(features, dtype='float', padding='post', truncating='post')
    return features, labels

def tf_function_wrapper(features, labels):
    return tf.py_function(func=process_path, inp=[features['filename'], labels], Tout=tf.float32), labels

librivox_paths_ds = tf.data.experimental.make_csv_dataset(
    "librivox-test-clean-wav.csv", batch_size=4,
    shuffle=False, label_name="transcript")

librivox_ds = librivox_paths_ds.map(tf_function_wrapper)

for feature_batch, labels in librivox_ds.take(1):
    tf.print(feature_batch)

    

[[[0.000732421875]
  [0.000823974609]
  [0.00173950195]
  ...
  [0]
  [0]
  [0]]

 [[-0.00778198242]
  [-0.00921630859]
  [-0.00909423828]
  ...
  [-0.00262451172]
  [-0.00671386719]
  [-0.0113830566]]]
